In [3]:
import pandas as pd

file_path = '../data/All-Purpose Cleaners.xlsx'

# Read the Excel file
df = pd.read_excel(file_path)

# Now df is a DataFrame containing the data from your Excel file

df_dict = df.to_dict()

df.to_csv('../data/All-Purpose Cleaners.csv', index=False)


In [5]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv(
    dotenv_path='../ops/.env',
)

api_key = os.getenv('OPENAI_API_KEY')

client = OpenAI(api_key=api_key)



In [7]:
file = client.files.create(
  file=open("../data/All-Purpose Cleaners.csv", "rb"),
  purpose='assistants'
)

print(file.id)

file-vIYUwwf8WY5a9MHUBNDSrqxV


In [15]:
assistant = client.beta.assistants.create(
  instructions="You are a data analyst assistant when given a requirement and a csv file analyse the csv file and generate a report with the requirements and number of rows that satisfies the requirement for each requirements.",
  model="gpt-4-1106-preview",
  tools=[{"type": "code_interpreter"}],
  file_ids=[file.id]
)

In [16]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": """This is the requirement. 1. Product name must include the brand name, actual name and size of the product as part
of the overall name e.g. “LaxmiBhog atta 1KG bag” 2. Every product must have high quality images (preferably 500x500 px) 3. Each product must have meaningful and related short description column and full description column associated
with the product 4. Slab options will be shown in a separate slab table for those products who have more than one pricing slab 5. All variants of a product will be shown in a variants section sorted by variant name in a horizontal scrollable section
""", "file_ids": [file.id]
        }
    ]
)

In [17]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

print(run.status)

queued


In [24]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

print(run.status)

completed


In [25]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    )

print(messages.data[0].content[0].text.value)

The count of rows satisfying each requirement is as follows:

- **Requirement 1** (Product name includes brand, actual name, and size): 580 rows meet this condition.
- **Requirement 2** (Product has high-quality images): 580 rows have a URL in the 'Images' column, which we're assuming indicates the presence of an image.
- **Requirement 3** (Each product has meaningful and related short and full descriptions): Only 1 row meets this condition based on our assumptions about the 'Features' and 'About' columns.
- **Requirement 4** (Slab options in a separate table for products with multiple pricing slabs): We cannot validate this requirement with the given data, so the count is 0.
- **Requirement 5** (All variants of a product are shown): Again, we cannot validate this requirement with the given data, so the count is 0.

Here's a random sample of image URLs from the 'Images' column for your reference:

```
315    https://datalabs.siva3.io/image/Eunimart_groce...
268    https://datalabs.siva